# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
from azureml.core.compute_target import ComputeTargetException

# Choose a name for your CPU cluster
compute_cluster_name = "cpucluster"

# Verify that cluster does not exist already
try:
    compute_cluster = ComputeTarget(workspace=ws, name=compute_cluster_name) 
    print('Found existing cluster, use it.')
except ComputeTargetException:
    # To use a different region for the compute, add a location='<region>' parameter
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    compute_cluster = ComputeTarget.create(ws, compute_cluster_name, compute_config)

compute_cluster.wait_for_completion(show_output=True)

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
from azureml.core.compute_target import ComputeTargetException

# Choose a name for your CPU cluster
compute_cluster_name = "cpucluster"

# Verify that cluster does not exist already
try:
    compute_cluster = ComputeTarget(workspace=ws, name=compute_cluster_name) 
    print('Found existing cluster, use it.')
except ComputeTargetException:
    # To use a different region for the compute, add a location='<region>' parameter
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    compute_cluster = ComputeTarget.create(ws, compute_cluster_name, compute_config)

compute_cluster.wait_for_completion(show_output=True)

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [ ]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

# choose a name for experiment
experiment_name = 'palsy'
project_folder = './palsy-project'


experiment=Experiment(ws, experiment_name)
print(experiment)

# Try to load the dataset from the Workspace. Otherwise, create it from the file
# NOTE: update the key to match the dataset name
found = False
key = "bellpalsy"
description_text = "Datapoints of face"

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 

if not found:
        # Create AML Dataset and register it into Workspace
#         datastore = Datastore.get(ws, datastore_name='palda')
#         example_data=datastore
        path="https://raw.githubusercontent.com/nsourlos/palda_azure/main/palda.csv"
#         example_data = 'https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv'
        dataset = Dataset.Tabular.from_delimited_files(path=path)#path = [(datastore, 'palda.csv')])#example_data)        
        #Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description=description_text)

df = df.iloc[: , :-1]

df = dataset.to_pandas_dataframe()
df.describe()

## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [ ]:
from azureml.train.hyperdrive.runconfig import HyperDriveConfig

# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = BanditPolicy(slack_factor=0.1,evaluation_interval=2)

#TODO: Create the different params that you will be using during training
param_sampling = RandomParameterSampling( {
       "--C": choice (0.01,0.1,1),
       "--max_iter": choice(4,8,16,32,64,128,256) #choice(10,20,40,60,100,150,200)
   })

#TODO: Create your estimator and hyperdrive config
estimator = SKLearn( source_directory=script_folder,compute_target=compute_cluster,entry_script="train.py")

hyperdrive_run_config = HyperDriveConfig(hyperparameter_sampling=ps, primary_metric_name='Accuracy',primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,max_total_runs=20,policy=policy, estimator=est)
#max_concurrent_runs=5

In [ ]:
#TODO: Submit your experiment
from azureml.widgets import RunDetails

hyperdrive_run=exp.submit(config=hyperdrive_config) 
RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output=True)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [ ]:
from azureml.widgets import RunDetails
RunDetails(remote_run).show()
remote_run.wait_for_completion(show_output=True)

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [ ]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
best_run=hyperdrive_run.get_best_run_by_primary_metric()
# if an error occurs it suggests that the best_run has the value equivalence of null in other languages, which means 
#it does not contain anything. This means that the there was no result from hyperdrive_run.get_best_run_by_primary_metric() 
#as used in the notebook to retrieve the best run, and thus nothing was assigned the best_run object. 
#The question then would be: did the training complete successfully?

print("best run metrics :", hyperdrive_best_run.get_metrics())
#we can also use 'hyperdrive_best_run.get_details()' and 'hyperdrive_best_run.get_file_names()'

model = best_run.register_model(model_name = 'best_model', model_path = 'outputs/model.joblib')
#Taken from https://github.com/Azure/MachineLearningNotebooks/issues/269

In [ ]:
#TODO: Save the best model
run.get_metrics()

import onnxmltools

best_run2, fitted_model = run.get_output(return_onnx_model=True)
# Taken from https://docs.microsoft.com/en-us/azure/machine-learning/tutorial-auto-train-models

onnxmltools.utils.save_model(fitted_model, './outputs/best_hyper_drive.onnx')

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [ ]:
model=best_run.register_model(model_name='hyperdrive_model',model_path='./outputs/model.joblib')

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.

